#### Import the libraries

In [2]:
!pip install optuna


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install dataframe_image


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pycaret


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dataframe_image as dfi

from pycaret.classification import *

In [6]:
df = pd.read_csv('./out.csv', index_col=0)
#df_type = df['Type']
df.shape

(17833, 193)

#### Remove duplicates

In [7]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

136

In [8]:
df.columns.values

array(['commit_hash', 'start_line', 'end_line', 'smell', 'id',
       'reviewer_id', 'sample_id', 'severity', 'longmethod_label',
       'featureenvy_label', 'dataclass_label', 'blob_label',
       'agreement_experts', 'agreement_smell_type', 'agreement_smell',
       'system_name', 'review_timestamp', 'type', 'code_name',
       'repository', 'path', 'link', 'is_from_industry_relevant_project',
       'weighted_average', 'qtd_rewier_id', 'Organic_fileRelativePath',
       'Organic_startLine', 'Organic_endLine', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion', 'Organic_IsAbstract',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'smell_Organic',
       'Organic_smellReason', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements',
       'Organic_Nu

In [9]:
df = df.drop('start_line',1)
df = df.drop('end_line',1)
df = df.drop('id',1)
df = df.drop('reviewer_id',1)
df = df.drop('smell',1)
df = df.drop('sample_id',1)
df = df.drop('severity',1)
df = df.drop('review_timestamp',1)
df = df.drop('code_name', 1)
df = df.drop('repository', 1)
df = df.drop('path',1)
df = df.drop('link',1)
df = df.drop('weighted_average', 1)
df = df.drop('qtd_rewier_id', 1)
df = df.drop('is_from_industry_relevant_project',1)
df = df.drop('Organic_fileRelativePath', 1)
df = df.drop('Organic_startLine', 1)
df = df.drop('Organic_endLine', 1)
df = df.drop('PMD_Problem', 1)
df = df.drop('PMD_Package', 1)
df = df.drop('PMD_File', 1)
df = df.drop('PMD_Priority', 1)
df = df.drop('PMD_Line', 1)
df = df.drop('agreement_smell_type', 1)
df = df.drop('agreement_smell', 1)
#df = df.drop('PMD_Rule', 1)
df = df.drop('implementation_Project Name', 1)
df = df.drop('implementation_code_name', 1)
df = df.drop('implementation_Type Name', 1)
df = df.drop('implementation_Method Name', 1)
df = df.drop('implementation_Method start line no', 1)
df = df.drop('CK_Class_file', 1)
df = df.drop('CK_Class_class', 1)
df = df.drop('Designite_Project Name',1)
df = df.drop('Designite_Package Name',1)
df = df.drop('Unnamed: 0',1)



## Drop Code Smell - LongMethod

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

In [36]:
#consider only the column 'longmethod_label'

df_longMethod = df.drop(columns=['featureenvy_label','dataclass_label','blob_label'])
df_longMethod.columns

Index(['commit_hash', 'longmethod_label', 'agreement_experts', 'system_name',
       'type', 'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=155)

#### Check the correlation and remove the high-correlated features from the dataset

In [37]:
corr = df_longMethod.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_longMethod.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_longMethod.columns[columns]
high_corr = set(df_longMethod.columns) - set(selected_columns)
df_longMethod = df_longMethod[selected_columns]
df_longMethod.columns

Index(['longmethod_label', 'agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [38]:
results = df_longMethod.longmethod_label.values
df_longMethod = df_longMethod.drop('longmethod_label', axis=1)
df_longMethod['longmethod_label'] = results
df_longMethod.columns
display(df_longMethod.dtypes)

agreement_experts                                       int64
Organic_WeighOfClass                                  float64
Organic_NumberOfAccessorMethods                       float64
Organic_ClassEffectiveLinesOfCode                     float64
Organic_LCOM3                                         float64
Organic_TightClassCohesion                            float64
Organic_PublicFieldCount                              float64
Organic_OverrideRatio                                 float64
Organic_WeightedMethodCount                           float64
Organic_ChangingMethods                               float64
Organic_NumberOfTryStatements                         float64
Organic_MaxNesting                                    float64
Organic_NumberOfFinallyStatements                     float64
Organic_NumberOfThrowStatements                       float64
Organic_NumberOfTryStatementsWithNoCatchAndFinally    float64
Organic_CouplingIntensity                             float64
Organic_

#### Setup the model creation with pycaret

In [39]:
s = setup(data=df_longMethod, target='longmethod_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 26109, number of negative: 26109
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3988
[LightGBM] [Info] Number of data points in the train set: 52218, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,4778
1,Target,longmethod_label
2,Target type,Binary
3,Original data shape,"(40951, 53)"
4,Transformed data shape,"(64504, 11)"
5,Transformed train set shape,"(52218, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,52
8,Rows with missing values,100.0%
9,Preprocess,True


In [40]:
df = get_config('X')
df

,agreement_experts,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,Organic_WeightedMethodCount,Organic_ChangingMethods,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,1,0.542857,16.0,160.0,0.900735,0.139496,0.0,0.333333,57.0,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [43]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8792,0.9627,0.9742,0.4237,0.5902,0.5320,0.5959,0.6290
dt,Decision Tree Classifier,0.8785,0.9617,0.9754,0.4228,0.5895,0.5311,0.5954,0.4130
rf,Random Forest Classifier,0.8782,0.9621,0.9742,0.4220,0.5885,0.5300,0.5943,0.7450
xgboost,Extreme Gradient Boosting,0.8742,0.9596,0.9699,0.4135,0.5795,0.5193,0.5850,0.4870
knn,K Neighbors Classifier,0.9282,0.9206,0.5239,0.6155,0.5656,0.5268,0.5290,0.5820
lightgbm,Light Gradient Boosting Machine,0.8644,0.9575,0.9695,0.3952,0.5612,0.4974,0.5679,0.5460
gbc,Gradient Boosting Classifier,0.7590,0.9172,0.9668,0.2662,0.4174,0.3226,0.4298,1.1430
ada,Ada Boost Classifier,0.7469,0.9041,0.9491,0.2546,0.4013,0.3033,0.4093,0.7060
lda,Linear Discriminant Analysis,0.6128,0.8530,0.9992,0.1872,0.3152,0.1942,0.3277,0.3900
ridge,Ridge Classifier,0.6121,0.0000,0.9992,0.1869,0.3149,0.1937,0.3272,0.3810


#### Tune the models

In [53]:
df2=pull()
dfi.export(df2, 'LongMethod_RQ_3_1.png')

In [54]:
tuned_top5= [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7241,0.9186,1.0000,0.2445,0.3929,0.2912,0.4128
1,0.7283,0.9177,0.9883,0.2459,0.3938,0.2926,0.4108
2,0.7527,0.9205,0.9883,0.2638,0.4165,0.3207,0.4339
3,0.7426,0.9320,0.9922,0.2566,0.4077,0.3098,0.4260
4,0.7349,0.9267,1.0000,0.2520,0.4025,0.3031,0.4226
5,0.7404,0.9245,0.9922,0.2543,0.4048,0.3066,0.4234
6,0.7258,0.9257,0.9882,0.2435,0.3907,0.2892,0.4080
7,0.7348,0.9297,0.9922,0.2502,0.3997,0.3002,0.4182
8,0.7355,0.9229,0.9961,0.2512,0.4013,0.3021,0.4207


[I 2024-01-18 19:54:07,160] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 20:00:10,003] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8354,0.9445,0.9531,0.3466,0.5083,0.4342,0.5148
1,0.7858,0.9234,0.9531,0.2884,0.4428,0.3543,0.4518
2,0.8465,0.9446,0.9609,0.3639,0.5279,0.4576,0.5349
3,0.8594,0.9477,0.9492,0.3839,0.5467,0.4806,0.5499
4,0.7949,0.9278,0.9766,0.3005,0.4596,0.3741,0.4735
5,0.8210,0.9414,0.9725,0.3289,0.4916,0.4136,0.5035
6,0.8280,0.9429,0.9843,0.3392,0.5045,0.4289,0.5185
7,0.7987,0.9401,0.9922,0.3056,0.4672,0.3833,0.4849
8,0.8077,0.9379,0.9961,0.3159,0.4797,0.3984,0.4977


[I 2024-01-18 20:00:39,488] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 20:02:41,637] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8619,0.9615,1.0000,0.3926,0.5639,0.4997,0.5771
1,0.8542,0.9620,1.0000,0.3798,0.5505,0.4837,0.5648
2,0.8779,0.9631,0.9922,0.4219,0.5921,0.5336,0.6014
3,0.8825,0.9638,1.0000,0.4317,0.6031,0.5465,0.6132
4,0.8556,0.9591,0.9961,0.3817,0.5519,0.4855,0.5652
5,0.8674,0.9621,0.9961,0.4013,0.5721,0.5099,0.5840
6,0.8604,0.9573,0.9961,0.3890,0.5595,0.4948,0.5724
7,0.8573,0.9664,0.9961,0.3837,0.5540,0.4882,0.5673
8,0.8601,0.9634,1.0000,0.3887,0.5598,0.4951,0.5736


[I 2024-01-18 20:09:07,253] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 20:12:22,612] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9327,0.9620,0.6953,0.6075,0.6485,0.6114,0.6131
1,0.9292,0.9609,0.6836,0.5892,0.6329,0.5940,0.5960
2,0.9369,0.9638,0.6875,0.6354,0.6604,0.6257,0.6263
3,0.9278,0.9629,0.6367,0.5884,0.6116,0.5719,0.5724
4,0.9229,0.9528,0.6133,0.5627,0.5869,0.5445,0.5451
5,0.9295,0.9628,0.6549,0.5943,0.6231,0.5844,0.5852
6,0.9257,0.9590,0.6314,0.5750,0.6019,0.5610,0.5617
7,0.9295,0.9633,0.7059,0.5863,0.6406,0.6019,0.6051
8,0.9330,0.9645,0.6941,0.6082,0.6484,0.6115,0.6131


[I 2024-01-18 20:12:40,286] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 20:15:14,473] Finished hyperparameter search!


#### Ensemble the top-5 models

In [55]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8835,0.9648,0.9531,0.4311,0.5937,0.5367,0.5945
1,0.8720,0.9634,0.9805,0.4095,0.5777,0.5168,0.5855
2,0.8943,0.9671,0.9609,0.4564,0.6189,0.5664,0.6194
3,0.8995,0.9664,0.9492,0.4691,0.6279,0.5774,0.6255
4,0.8786,0.9601,0.9570,0.4210,0.5847,0.5259,0.5870
5,0.8876,0.9642,0.9647,0.4401,0.6044,0.5494,0.6070
6,0.8789,0.9603,0.9725,0.4218,0.5884,0.5300,0.5939
7,0.8859,0.9680,0.9804,0.4371,0.6046,0.5491,0.6105
8,0.8723,0.9605,0.9843,0.4095,0.5783,0.5177,0.5872


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [56]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9344,0.9657,0.6916,0.6183,0.6529,0.6168,0.6180


In [57]:
result = pull()

#### Plot the ensemble model

In [58]:
result.to_csv('models.csv')

#### Finalize and save the ensemble model

In [59]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['agreement_experts',
                                              'Organic_WeighOfClass',
                                              'Organic_NumberOfAccessorMethods',
                                              'Organic_ClassEffectiveLinesOfCode',
                                              'Organic_LCOM3',
                                              'Organic_TightClassCohesion',
                                              'Organic_PublicFieldCount',
                                              'Organic_OverrideRatio',
                                              'Organic_WeightedMethodCount',
                                              'Org...
                                                                 norm_order=1,
                                                                 prefit=Fal

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

## Drop Code Smell - FeatureEnvy

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the  Feature Envy smell. To train other models, remove the other code smells.

In [10]:
#consider only the column 'featureenvy_label'
df.columns.values
df_featureenvy = df.drop(columns=['longmethod_label','dataclass_label','blob_label'])
df_featureenvy.columns

Index(['commit_hash', 'featureenvy_label', 'agreement_experts', 'system_name',
       'type', 'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=155)

#### Check the correlation and remove the high-correlated features from the dataset

In [11]:
corr = df_featureenvy.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_featureenvy.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_featureenvy.columns[columns]
high_corr = set(df_featureenvy.columns) - set(selected_columns)
df_featureenvy = df_featureenvy[selected_columns]
df_featureenvy.columns

Index(['featureenvy_label', 'agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_Al

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [12]:
results = df_featureenvy.featureenvy_label.values
df_featureenvy = df_featureenvy.drop('featureenvy_label', axis=1)
df_featureenvy['featureenvy_label'] = results
df_featureenvy.columns

Index(['agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All', 'cluster_Gaussian

#### Setup the model creation with pycaret

In [13]:
s = setup(data=df_featureenvy, target='featureenvy_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 26088, number of negative: 26088
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2151
[LightGBM] [Info] Number of data points in the train set: 52176, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,7268
1,Target,featureenvy_label
2,Target type,Binary
3,Original data shape,"(40951, 53)"
4,Transformed data shape,"(64462, 11)"
5,Transformed train set shape,"(52176, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,52
8,Rows with missing values,100.0%
9,Preprocess,True


In [14]:
df = get_config('X')
df

,agreement_experts,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,Organic_WeightedMethodCount,Organic_ChangingMethods,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [15]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8753,0.9635,0.9837,0.4182,0.5868,0.5271,0.5943,0.6590
dt,Decision Tree Classifier,0.8740,0.9625,0.9852,0.4156,0.5846,0.5244,0.5925,0.3930
rf,Random Forest Classifier,0.8740,0.9633,0.9849,0.4157,0.5845,0.5244,0.5924,0.8720
knn,K Neighbors Classifier,0.9286,0.9299,0.5316,0.6201,0.5720,0.5334,0.5355,0.6160
xgboost,Extreme Gradient Boosting,0.8658,0.9606,0.9880,0.4004,0.5697,0.5066,0.5795,0.6040
lightgbm,Light Gradient Boosting Machine,0.8565,0.9585,0.9887,0.3846,0.5537,0.4873,0.5647,0.6220
gbc,Gradient Boosting Classifier,0.7616,0.9080,0.9659,0.2697,0.4216,0.3270,0.4331,1.2390
ada,Ada Boost Classifier,0.7443,0.8943,0.9558,0.2546,0.4021,0.3031,0.4109,0.8580
lda,Linear Discriminant Analysis,0.6773,0.8562,0.9550,0.2124,0.3474,0.2349,0.3526,0.3890
ridge,Ridge Classifier,0.6775,0.0000,0.9480,0.2116,0.3459,0.2332,0.3492,0.5420


In [16]:
df2=pull()
dfi.export(df2, 'FeatureEnvy_RQ_3_1.png')

#### Tune the models

In [17]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7789,0.9473,1.0000,0.2892,0.4487,0.3592,0.4679
1,0.7377,0.9161,0.9961,0.2550,0.4060,0.3066,0.4245
2,0.7558,0.9176,0.9961,0.2688,0.4234,0.3282,0.4420
3,0.7468,0.9323,1.0000,0.2622,0.4155,0.3182,0.4350
4,0.7510,0.9241,0.9961,0.2649,0.4186,0.3222,0.4372
5,0.7435,0.9177,0.9883,0.2576,0.4087,0.3106,0.4257
6,0.7502,0.9216,0.9961,0.2636,0.4169,0.3206,0.4359
7,0.7449,0.9142,1.0000,0.2601,0.4129,0.3154,0.4327
8,0.7603,0.9288,1.0000,0.2730,0.4289,0.3349,0.4484


[I 2024-01-18 21:59:12,952] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:05:19,778] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8375,0.9488,1.0000,0.3564,0.5255,0.4529,0.5410
1,0.8291,0.9375,0.9806,0.3428,0.5080,0.4323,0.5202
2,0.8218,0.9294,0.9651,0.3316,0.4936,0.4152,0.5029
3,0.8273,0.9432,0.9845,0.3409,0.5065,0.4303,0.5196
4,0.8235,0.9430,0.9961,0.3373,0.5039,0.4269,0.5199
5,0.8367,0.9453,0.9650,0.3508,0.5145,0.4410,0.5230
6,0.8116,0.9369,0.9922,0.3216,0.4857,0.4051,0.5020
7,0.8269,0.9398,0.9844,0.3396,0.5050,0.4288,0.5184
8,0.8465,0.9531,0.9806,0.3677,0.5349,0.4648,0.5453


[I 2024-01-18 22:05:53,894] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:08:05,286] Finished hyperparameter search!


...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9292,0.9607,0.6357,0.6007,0.6177,0.5787,0.5790
1,0.9281,0.9616,0.6667,0.5890,0.6255,0.5859,0.5873
2,0.9327,0.9628,0.6744,0.6148,0.6433,0.6062,0.6070
3,0.9323,0.9614,0.6628,0.6151,0.6381,0.6008,0.6013
4,0.9302,0.9612,0.6163,0.6115,0.6139,0.5756,0.5756
5,0.9285,0.9503,0.6498,0.5922,0.6197,0.5803,0.5810
6,0.9351,0.9640,0.6732,0.6291,0.6504,0.6147,0.6151
7,0.9351,0.9646,0.6498,0.6350,0.6423,0.6066,0.6067
8,0.9330,0.9661,0.6938,0.6130,0.6509,0.6140,0.6155


[I 2024-01-18 22:15:28,292] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:17:55,997] Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8811,0.9665,1.0000,0.4307,0.6021,0.5448,0.6119
1,0.8699,0.9626,1.0000,0.4089,0.5804,0.5190,0.5920
2,0.8601,0.9571,1.0000,0.3915,0.5627,0.4977,0.5756
3,0.8657,0.9656,0.9961,0.4009,0.5717,0.5087,0.5831
4,0.8640,0.9598,1.0000,0.3981,0.5695,0.5059,0.5819
5,0.8636,0.9616,0.9883,0.3956,0.5651,0.5012,0.5754
6,0.8695,0.9657,1.0000,0.4073,0.5788,0.5173,0.5907
7,0.8706,0.9647,1.0000,0.4092,0.5808,0.5197,0.5925
8,0.8699,0.9661,1.0000,0.4089,0.5804,0.5190,0.5919


[I 2024-01-18 22:18:08,922] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:21:17,507] Finished hyperparameter search!


#### Ensemble the top-5 models

In [18]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8842,0.9668,1.0000,0.4373,0.6085,0.5524,0.6178
1,0.8702,0.9599,0.9884,0.4087,0.5782,0.5167,0.5873
2,0.8643,0.9603,0.9884,0.3978,0.5673,0.5036,0.5772
3,0.8692,0.9656,0.9961,0.4073,0.5782,0.5164,0.5890
4,0.8692,0.9618,0.9922,0.4070,0.5772,0.5154,0.5873
5,0.8650,0.9593,0.9728,0.3968,0.5637,0.5000,0.5706
6,0.8761,0.9677,0.9883,0.4191,0.5886,0.5294,0.5971
7,0.8772,0.9663,0.9805,0.4207,0.5888,0.5298,0.5956
8,0.8737,0.9674,0.9961,0.4159,0.5868,0.5266,0.5969


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [19]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9342,0.9643,0.6576,0.6275,0.6422,0.6060,0.6062


In [20]:
result = pull()

#### Plot the ensemble model

In [21]:
result.to_csv('models_featureenvy.csv')

#### Finalize and save the ensemble model

In [22]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_featureenvy')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['agreement_experts',
                                              'Organic_WeighOfClass',
                                              'Organic_NumberOfAccessorMethods',
                                              'Organic_ClassEffectiveLinesOfCode',
                                              'Organic_LCOM3',
                                              'Organic_TightClassCohesion',
                                              'Organic_PublicFieldCount',
                                              'Organic_OverrideRatio',
                                              'Organic_WeightedMethodCount',
                                              'Org...
                                                                 norm_order=1,
                                                                 prefit=Fal

#### Interpret the ensemble model

In [23]:
!pip install shap
interpret_model(tuned_top5[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Drop Code Smell - DataClass

In [114]:
##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the DataClass smell. To train other models, remove the other code smells.

In [28]:
#consider only the column 'dataclass_label'

df_dataclass = df.drop(columns=['longmethod_label','featureenvy_label','blob_label'])
df_dataclass.columns

Index(['commit_hash', 'dataclass_label', 'agreement_experts', 'system_name',
       'type', 'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=155)

#### Check the correlation and remove the high-correlated features from the dataset

In [29]:
corr = df_dataclass.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_dataclass.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_dataclass.columns[columns]
high_corr = set(df_dataclass.columns) - set(selected_columns)
df_dataclass = df_dataclass[selected_columns]
df_dataclass.columns

Index(['dataclass_label', 'agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All'

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of DataClass.

In [30]:
results = df_dataclass.dataclass_label.values
df_dataclass = df_dataclass.drop('dataclass_label', axis=1)
df_dataclass['dataclass_label'] = results
df_dataclass.columns

Index(['agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All', 'cluster_Gaussian

#### Setup the model creation with pycaret

In [31]:
s = setup(data=df_dataclass, target='dataclass_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23277, number of negative: 23277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1676
[LightGBM] [Info] Number of data points in the train set: 46554, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,1565
1,Target,dataclass_label
2,Target type,Binary
3,Original data shape,"(40951, 53)"
4,Transformed data shape,"(58840, 11)"
5,Transformed train set shape,"(46554, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,52
8,Rows with missing values,100.0%
9,Preprocess,True


In [32]:
df = get_config('X')
df

,agreement_experts,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,Organic_WeightedMethodCount,Organic_ChangingMethods,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,1.0,0.0,233.0,0.894309,0.010753,0.0,0.833333,76.0,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [33]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.8663,0.9625,0.9640,0.5890,0.7309,0.6489,0.6840,0.3530
rf,Random Forest Classifier,0.8651,0.9619,0.9634,0.5868,0.7290,0.6462,0.6817,0.6060
et,Extra Trees Classifier,0.8632,0.9599,0.9608,0.5829,0.7254,0.6415,0.6772,0.5350
xgboost,Extreme Gradient Boosting,0.8561,0.9573,0.9612,0.5698,0.7153,0.6273,0.6658,0.4950
knn,K Neighbors Classifier,0.8925,0.9291,0.6806,0.7296,0.7040,0.6385,0.6392,0.5120
lightgbm,Light Gradient Boosting Machine,0.8454,0.9572,0.9664,0.5508,0.7016,0.6076,0.6514,0.6860
gbc,Gradient Boosting Classifier,0.7159,0.9050,0.9772,0.3965,0.5641,0.4049,0.4958,1.0310
ada,Ada Boost Classifier,0.7035,0.8836,0.9590,0.3847,0.5490,0.3835,0.4726,0.6200
lda,Linear Discriminant Analysis,0.6592,0.8466,0.9983,0.3558,0.5245,0.3421,0.4535,0.3410
ridge,Ridge Classifier,0.6534,0.0000,0.9985,0.3518,0.5202,0.3354,0.4482,0.3270


In [34]:
df2=pull()
dfi.export(df2, 'DataClass_RQ_3_1.png')

#### Tune the models

In [35]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8172,0.9396,0.9666,0.5073,0.6654,0.5559,0.6105
1,0.8211,0.9362,0.9481,0.5131,0.6658,0.5579,0.6070
2,0.8165,0.9332,0.9351,0.5065,0.6571,0.5465,0.5944
3,0.8106,0.9412,0.9592,0.4981,0.6557,0.5424,0.5979
4,0.8242,0.9381,0.9351,0.5180,0.6667,0.5603,0.6053
5,0.8140,0.9316,0.9294,0.5025,0.6523,0.5403,0.5879
6,0.8119,0.9332,0.9628,0.4995,0.6578,0.5454,0.6012
7,0.8077,0.9380,0.9703,0.4943,0.6550,0.5405,0.5995
8,0.8133,0.9335,0.9555,0.5019,0.6581,0.5463,0.5998


[I 2024-01-18 22:38:56,630] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:41:02,320] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7681,0.9332,0.9870,0.4471,0.6154,0.4811,0.5585
1,0.7935,0.9317,0.9796,0.4761,0.6408,0.5191,0.5857
2,0.7987,0.9308,0.9647,0.4824,0.6432,0.5238,0.5849
3,0.7855,0.9338,0.9703,0.4661,0.6297,0.5037,0.5708
4,0.7907,0.9393,0.9722,0.4725,0.6359,0.5126,0.5783
5,0.7872,0.9351,0.9926,0.4684,0.6365,0.5120,0.5842
6,0.7840,0.9342,0.9796,0.4643,0.6300,0.5036,0.5736
7,0.7777,0.9299,0.9870,0.4578,0.6255,0.4960,0.5701
8,0.8022,0.9362,0.9703,0.4870,0.6485,0.5310,0.5922


[I 2024-01-18 22:41:14,813] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:47:33,220] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8130,0.9572,1.0000,0.5014,0.6679,0.5570,0.6213
1,0.8322,0.9569,1.0000,0.5284,0.6915,0.5908,0.6475
2,0.8336,0.9526,0.9963,0.5306,0.6925,0.5925,0.6477
3,0.8211,0.9538,1.0000,0.5124,0.6776,0.5709,0.6320
4,0.8333,0.9611,1.0000,0.5300,0.6928,0.5927,0.6490
5,0.8280,0.9579,1.0000,0.5218,0.6858,0.5829,0.6413
6,0.8248,0.9600,1.0000,0.5173,0.6819,0.5773,0.6370
7,0.8255,0.9563,1.0000,0.5188,0.6831,0.5788,0.6382
8,0.8430,0.9610,1.0000,0.5450,0.7055,0.6107,0.6630


[I 2024-01-18 22:53:34,594] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:56:33,728] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8926,0.9604,0.8404,0.6711,0.7463,0.6792,0.6862
1,0.8894,0.9571,0.8071,0.6713,0.7329,0.6640,0.6684
2,0.8884,0.9554,0.7885,0.6735,0.7265,0.6569,0.6601
3,0.8947,0.9592,0.8237,0.6820,0.7462,0.6805,0.6853
4,0.9083,0.9703,0.8219,0.7262,0.7711,0.7140,0.7162
5,0.8932,0.9579,0.7881,0.6883,0.7348,0.6684,0.6708
6,0.8960,0.9608,0.8401,0.6807,0.7521,0.6872,0.6933
7,0.8901,0.9590,0.8330,0.6662,0.7403,0.6717,0.6784
8,0.8908,0.9614,0.7978,0.6782,0.7332,0.6651,0.6685


[I 2024-01-18 22:56:52,461] Searching the best hyperparameters using 28665 samples...
[I 2024-01-18 22:59:27,056] Finished hyperparameter search!


#### Ensemble the top-5 models

In [36]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8751,0.9721,0.9666,0.6051,0.7443,0.6674,0.6994
1,0.8905,0.9688,0.9406,0.6426,0.7636,0.6955,0.7169
2,0.8640,0.9565,0.9276,0.5875,0.7194,0.6355,0.6644
3,0.8559,0.9583,0.9481,0.5703,0.7122,0.6239,0.6597
4,0.8793,0.9716,0.9573,0.6150,0.7489,0.6744,0.7030
5,0.8716,0.9602,0.9721,0.5970,0.7397,0.6609,0.6955
6,0.8699,0.9622,0.9628,0.5947,0.7353,0.6553,0.6887
7,0.8615,0.9608,0.9703,0.5785,0.7249,0.6401,0.6783
8,0.8936,0.9730,0.9573,0.6466,0.7719,0.7058,0.7289


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [37]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.8924,0.9592,0.8208,0.6762,0.7415,0.6744,0.6794


In [38]:
result = pull()

#### Plot the ensemble model

In [39]:
result.to_csv('models_dataclass.csv')

In [43]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_dataclass')

IndexError: Length of values (66506) does not match length of index (40951). This usually happens when transformations that drop rows aren't applied on all the columns.

#### Interpret the ensemble model

In [42]:
!pip install shap
interpret_model(tuned_top5[0], save=True)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Drop Code Smell - Blob

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Blob. To train other models, remove the other code smells.

In [10]:
#consider only the column 'blob_label'

df_blob = df.drop(columns=['featureenvy_label','dataclass_label','longmethod_label'])
df_blob.columns

Index(['commit_hash', 'blob_label', 'agreement_experts', 'system_name', 'type',
       'Organic_WeighOfClass', 'Organic_NumberOfAccessorMethods',
       'Organic_ClassEffectiveLinesOfCode', 'Organic_LCOM3',
       'Organic_TightClassCohesion',
       ...
       'smell_Organic_blob', 'smell_PMD_longmethod', 'smell_PMD_featureenvy',
       'smell_PMD_dataclass', 'smell_PMD_blob', 'smell_Designite_longmethod',
       'smell_Designite_featureenvy', 'smell_Designite_dataclass',
       'smell_Designite_blob', 'background'],
      dtype='object', length=155)

#### Check the correlation and remove the high-correlated features from the dataset

In [11]:
corr = df_blob.corr()
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_blob.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_blob.columns[columns]
high_corr = set(df_blob.columns) - set(selected_columns)
df_blob = df_blob[selected_columns]
df_blob.columns

Index(['blob_label', 'agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All', 'cl

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Blob.

In [12]:
results = df_blob.blob_label.values
df_blob = df_blob.drop('blob_label', axis=1)
df_blob['blob_label'] = results
df_blob.columns

Index(['agreement_experts', 'Organic_WeighOfClass',
       'Organic_NumberOfAccessorMethods', 'Organic_ClassEffectiveLinesOfCode',
       'Organic_LCOM3', 'Organic_TightClassCohesion',
       'Organic_PublicFieldCount', 'Organic_OverrideRatio',
       'Organic_WeightedMethodCount', 'Organic_ChangingMethods',
       'Organic_NumberOfTryStatements', 'Organic_MaxNesting',
       'Organic_NumberOfFinallyStatements', 'Organic_NumberOfThrowStatements',
       'Organic_NumberOfTryStatementsWithNoCatchAndFinally',
       'Organic_CouplingIntensity', 'Organic_NumberOfAccessedVariables',
       'Organic_MethodEffectiveLinesOfCode', 'Organic_CouplingDispersion',
       'Organic_NumberOfDummyExceptionHandlers', 'Organic_ParameterCount',
       'Organic_ChangingClasses', 'Organic_NumberOfCatchStatements',
       'Organic_ThrownExceptionTypesCount', 'CK_Class_synchronizedFieldsQty',
       'Stars', 'Watching', 'Commits', 'LOC', 'Number of Contributors',
       'cluster_kmeans_All', 'cluster_Gaussian

#### Setup the model creation with pycaret

In [13]:
s = setup(data=df_blob, target='blob_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23652, number of negative: 23652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002482 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2052
[LightGBM] [Info] Number of data points in the train set: 47304, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,3509
1,Target,blob_label
2,Target type,Binary
3,Original data shape,"(40951, 53)"
4,Transformed data shape,"(59590, 11)"
5,Transformed train set shape,"(47304, 11)"
6,Transformed test set shape,"(12286, 11)"
7,Numeric features,52
8,Rows with missing values,100.0%
9,Preprocess,True


In [14]:
df = get_config('X')
df

,agreement_experts,Organic_WeighOfClass,Organic_NumberOfAccessorMethods,Organic_ClassEffectiveLinesOfCode,Organic_LCOM3,Organic_TightClassCohesion,Organic_PublicFieldCount,Organic_OverrideRatio,Organic_WeightedMethodCount,Organic_ChangingMethods,...,smell_Organic_dataclass,smell_Organic_blob,smell_PMD_longmethod,smell_PMD_featureenvy,smell_PMD_dataclass,smell_PMD_blob,smell_Designite_longmethod,smell_Designite_featureenvy,smell_Designite_dataclass,smell_Designite_blob
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,1.000000,0.0,29.0,0.00,1.000000,0.0,0.0,12.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,0.785714,3.0,114.0,0.85,0.296703,0.0,NaN,35.0,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [15]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8763,0.9679,0.9731,0.5888,0.7336,0.6593,0.6946,0.5160
dt,Decision Tree Classifier,0.8760,0.9678,0.9729,0.5884,0.7332,0.6587,0.6941,0.3490
rf,Random Forest Classifier,0.8749,0.9676,0.9747,0.5857,0.7316,0.6566,0.6928,0.6600
xgboost,Extreme Gradient Boosting,0.8685,0.9644,0.9697,0.5736,0.7207,0.6420,0.6799,0.4230
lightgbm,Light Gradient Boosting Machine,0.8574,0.9603,0.9731,0.5528,0.7049,0.6201,0.6633,0.4660
knn,K Neighbors Classifier,0.8909,0.9399,0.6916,0.6912,0.6901,0.6241,0.6249,0.5560
gbc,Gradient Boosting Classifier,0.7482,0.9158,0.9717,0.4079,0.5745,0.4354,0.5183,0.8880
ada,Ada Boost Classifier,0.7274,0.8997,0.9579,0.3874,0.5515,0.4028,0.4882,0.6050
ridge,Ridge Classifier,0.6477,0.0000,1.0000,0.3319,0.4983,0.3196,0.4361,0.3210
lda,Linear Discriminant Analysis,0.6455,0.8619,1.0000,0.3306,0.4968,0.3174,0.4342,0.3280


In [16]:
df2=pull()
dfi.export(df2, 'Blob_RQ_3_1.png')

#### Tune the models

In [17]:
tuned_top5= [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
  ]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7081,0.9155,1.0000,0.3744,0.5449,0.3897,0.4919
1,0.7203,0.9306,0.9960,0.3841,0.5544,0.4042,0.5019
2,0.7143,0.9214,0.9920,0.3793,0.5488,0.3957,0.4939
3,0.7171,0.9070,0.9960,0.3820,0.5522,0.4004,0.4989
4,0.7088,0.9143,0.9940,0.3749,0.5445,0.3891,0.4894
5,0.7170,0.9243,0.9860,0.3806,0.5492,0.3971,0.4930
6,0.7160,0.9143,0.9880,0.3799,0.5488,0.3963,0.4930
7,0.7080,0.9100,0.9940,0.3739,0.5434,0.3878,0.4884
8,0.7156,0.9138,0.9920,0.3800,0.5495,0.3971,0.4949


[I 2024-02-25 17:24:04,507] Searching the best hyperparameters using 28665 samples...
[I 2024-02-25 17:28:28,021] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8532,0.9571,0.9661,0.5450,0.6969,0.6097,0.6532
1,0.8507,0.9572,0.9820,0.5401,0.6969,0.6086,0.6565
2,0.8427,0.9576,0.9781,0.5274,0.6853,0.5926,0.6428
3,0.8483,0.9556,0.9661,0.5371,0.6904,0.6005,0.6459
4,0.8451,0.9583,0.9522,0.5323,0.6829,0.5910,0.6347
5,0.8472,0.9577,0.9621,0.5350,0.6876,0.5971,0.6421
6,0.8500,0.9569,0.9601,0.5398,0.6911,0.6020,0.6456
7,0.8329,0.9509,0.9800,0.5115,0.6721,0.5744,0.6291
8,0.8367,0.9524,0.9581,0.5178,0.6723,0.5761,0.6244


[I 2024-02-25 17:28:46,024] Searching the best hyperparameters using 28665 samples...
[I 2024-02-25 17:30:27,031] Finished hyperparameter search!


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8598,0.9648,0.9920,0.5553,0.7120,0.6289,0.6751
1,0.8716,0.9733,1.0000,0.5765,0.7314,0.6549,0.6977
2,0.8727,0.9691,0.9980,0.5792,0.7330,0.6570,0.6989
3,0.8580,0.9668,0.9980,0.5524,0.7111,0.6271,0.6753
4,0.8552,0.9668,0.9841,0.5483,0.7042,0.6184,0.6647
5,0.8639,0.9682,0.9980,0.5624,0.7194,0.6386,0.6844
6,0.8723,0.9691,0.9940,0.5784,0.7313,0.6550,0.6963
7,0.8590,0.9667,0.9980,0.5537,0.7123,0.6288,0.6766
8,0.8475,0.9594,0.9920,0.5344,0.6946,0.6048,0.6562


[I 2024-02-25 17:35:08,777] Searching the best hyperparameters using 28665 samples...
[I 2024-02-25 17:37:45,660] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8598,0.9555,0.9760,0.5563,0.7087,0.6253,0.6681
1,0.8845,0.9703,0.9820,0.6044,0.7483,0.6788,0.7125
2,0.8689,0.9586,0.9661,0.5746,0.7207,0.6421,0.6791
3,0.8723,0.9626,0.9582,0.5823,0.7244,0.6477,0.6817
4,0.8622,0.9659,0.9661,0.5620,0.7106,0.6283,0.6681
5,0.8664,0.9604,0.9721,0.5689,0.7178,0.6379,0.6772
6,0.8789,0.9669,0.9641,0.5948,0.7357,0.6628,0.6953
7,0.8688,0.9642,0.9561,0.5750,0.7181,0.6394,0.6745
8,0.8660,0.9647,0.9561,0.5696,0.7139,0.6336,0.6698


[I 2024-02-25 17:37:57,666] Searching the best hyperparameters using 28665 samples...
[I 2024-02-25 17:45:13,342] Finished hyperparameter search!


#### Ensemble the top-5 models

In [18]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8629,0.9579,0.9780,0.5619,0.7138,0.6321,0.6741
1,0.8863,0.9722,0.9780,0.6087,0.7504,0.6818,0.7140
2,0.8832,0.9688,0.9861,0.6015,0.7472,0.6769,0.7119
3,0.8744,0.9660,0.9801,0.5843,0.7321,0.6569,0.6943
4,0.8629,0.9674,0.9661,0.5633,0.7117,0.6298,0.6692
5,0.8810,0.9687,0.9701,0.5985,0.7403,0.6686,0.7014
6,0.8730,0.9634,0.9721,0.5818,0.7280,0.6518,0.6883
7,0.8719,0.9669,0.9840,0.5786,0.7288,0.6522,0.6916
8,0.8629,0.9599,0.9601,0.5632,0.7100,0.6280,0.6663


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


#### Plot the ensemble model

In [24]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8784,0.9674,0.9767,0.5923,0.7374,0.6644,0.6996


#### Finalize and save the ensemble model

In [25]:
result = pull()

In [26]:
result.to_csv('models_blob.csv')

In [28]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_blob1')

IndexError: Length of values (67580) does not match length of index (40951). This usually happens when transformations that drop rows aren't applied on all the columns.

#### Interpret the ensemble model

In [ ]:
!pip install shap
interpret_model(tuned_top5[0], save=True)